## Diet assignment - solution with PuLP optimization library

Data downloaded from: https://www.kaggle.com/omsaraksa/diet-for-linear-programing2020

Objective: minimize costs while fulfilling all constraints

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import pulp

In [4]:
data = pd.read_csv('data/diet.csv', index_col = 0)
data

,name,max_serv,size,energy,protein,calcium,magnesium,vitaminc,iron,price
index,,,,,,,,,,
1,Oatmeal,4,28,110,4,2,26,0,6,11
2,Chicken,3,100,205,32,12,1,2,1,16
3,Eggs,2,2,160,13,54,10,0,1,2
4,Whole Milk,8,237,160,8,285,0,0,11,8
5,Cherry pie,2,170,420,4,22,9,1,2,25
6,Pork with beans,2,206,260,14,80,31,1,1,4
7,Yogurt,3,3,58,10,110,11,0,0,10
8,Rice,5,30,500,10,28,2501,9,9,10
9,Meat,6,143,143,26,6,29,0,1,40


Constraints:
1. daily energy: min 1500 max 2000
2. each food selected only once
3. protein >= 50
4. calcium >= 200
5. magnesium >= 50
6. vitaminc >= 10
7. iron >= 10

In [34]:
_foods = data['name'].tolist()
_variables = pulp.LpVariable.dicts( 'food', _foods, lowBound = 0, upBound = 1, cat = pulp.const.LpInteger)

_costs = dict(zip(data['name'], data['price']))

_energy = dict(zip(data['name'], data['energy']))
_protein = dict(zip(data['name'], data['protein']))
_calcium = dict(zip(data['name'], data['calcium']))
_magnesium = dict(zip(data['name'], data['magnesium']))
_vitaminc = dict(zip(data['name'], data['vitaminc']))
_iron = dict(zip(data['name'], data['iron']))

In [35]:
prob = pulp.LpProblem(name = 'Diet optimization', sense = pulp.const.LpMinimize)

prob += pulp.lpSum(_variables[i] * _costs[i] for i in _foods)

prob += pulp.lpSum(_variables[i] * _energy[i] for i in _foods) <= 2000
prob += pulp.lpSum(_variables[i] * _energy[i] for i in _foods) >= 1500
prob += pulp.lpSum(_variables[i] * _protein[i] for i in _foods) >= 50
prob += pulp.lpSum(_variables[i] * _calcium[i] for i in _foods) >= 200
prob += pulp.lpSum(_variables[i] * _magnesium[i] for i in _foods) >= 50
prob += pulp.lpSum(_variables[i] * _vitaminc[i] for i in _foods) >= 10
prob += pulp.lpSum(_variables[i] * _iron[i] for i in _foods) >= 10

status = prob.solve()
print('Status:', pulp.LpStatus[status])


Status: Optimal


In [36]:
print('Selected foods:\n')

for variable in prob.variables():
    if variable.varValue == 1:
        print("{} = {}".format(variable.name, variable.varValue))

print('\n',  '=' * 50, '\n\nLeave out of diet:\n')

for variable in prob.variables():
    if variable.varValue == 0:
        print("{} = {}".format(variable.name, variable.varValue))

OPTIMUM = pulp.value(prob.objective)
print('\n',  '=' * 50)
print('\nOptimal cost for diet ($):', OPTIMUM)

Selected foods:

food_Eggs = 1.0
food_Oatmeal = 1.0
food_Pork_with_beans = 1.0
food_Potato = 1.0
food_Rice = 1.0
food_Whole_Milk = 1.0


Leave out of diet:

food_Cherry_pie = 0.0
food_Chicken = 0.0
food_Meat = 0.0
food_Yogurt = 0.0


Optimal cost for diet ($): 38.0
